<h1 align=center><font size = 5>Capstone Project: Segmenting and Clustering Neighborhoods in Toronto</font></h1>

In [2]:
import numpy as np

In [3]:
import csv
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
import pandas as pd
import bs4   # from http://beautiful-soup-4.readthedocs.io/en/latest/
from bs4 import BeautifulSoup
import requests
import lxml.html as lh
import urllib.request

def scrape_table_bs4(cname,cols):
    page  = urllib.request.urlopen(url).read()
    soup  = bs4.BeautifulSoup(page,'lxml')
    table = soup.find("table",class_=cname)
    header = [head.findAll(text=True)[0].strip() for head in table.find_all("th")]
    data   = [[td.findAll(text=True)[0].strip() for td in tr.find_all("td")]
              for tr in table.find_all("tr")]
    data    = [row for row in data if len(row) == cols]
    # Store data to this temporary dataframe
    raw_df = pd.DataFrame(data,columns=header)
    return raw_df

In [5]:
# Parsing using xpath
def scrape_table_lxml(XPATH,cols):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    table_content = doc.xpath(XPATH)
    for table in table_content:
        headers = [th.text_content().strip() for th in table.xpath('//th')]
        headers = headers[0:3]
        data    = [[td.text_content().strip() for td in tr.xpath('td')] 
                   for tr in table.xpath('//tbody/tr')]
        data    = [row for row in data if len(row) == cols]
        raw_df = pd.DataFrame(data,columns = headers)  
        return raw_df  # Transform the data in the table on Wikipedia into a pandas dataframe.

In [6]:
#Test in beautifulSoup
raw_NeighborhoodInToronto = scrape_table_bs4("wikitable",3)
print("# Neighborhoods in Toronto")
print(raw_NeighborhoodInToronto.info(verbose=True))

# Neighborhoods in Toronto
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 3 columns):
Postcode         288 non-null object
Borough          288 non-null object
Neighbourhood    288 non-null object
dtypes: object(3)
memory usage: 6.8+ KB
None


### Cleanup the Data as requested:
(1) Process cells with assigned borough only.
(2) More than one neighborhood can exist in one postal code area. They will be combined into one row with the neighborhoods separated with a comma.
(3) If a cell has a borough but not yet assigned a neighborhood, the neighborhood will be the same as the borough.

In [7]:
NeighborhoodInToronto = raw_NeighborhoodInToronto[~raw_NeighborhoodInToronto['Borough'].isin(['Not assigned'])]
NeighborhoodInToronto = NeighborhoodInToronto.sort_values(by=['Postcode','Borough','Neighbourhood'], ascending=[1,1,1]).reset_index(drop=True)
NeighborhoodInToronto.loc[NeighborhoodInToronto['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = NeighborhoodInToronto['Borough']
check_unassigned_post_state_sample = NeighborhoodInToronto.loc[NeighborhoodInToronto['Borough'] == 'Queen\'s Park']
NeighborhoodInToronto = NeighborhoodInToronto.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
NeighborhoodInToronto.head(12)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [8]:
NeighborhoodInToronto.shape

(103, 3)

In [10]:
NeighborhoodInToronto.to_csv('NeighborhoodInToronto.csv',index=False)